# Calidad de resultados segun k y alpha

## Pre - Experimentacion

In [1]:
# make cpp libraries
!cd ../../ && mkdir build
!cd ../../build/ && rm -rf *
!cd ../../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/lucas/.virtualenvs/metnum_tp2/bin/python (found version "3.8.10") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.8.so
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/lucas/Desktop/MetNum/TP/TP2/metnum_tp2
-- Configuring d

In [2]:
# verifico version de python e importo metnum
!pwd
!python --version

import sys
import os

p = os.path.abspath('../')
sys.path.insert(1, p)

import metnum

/home/lucas/Desktop/MetNum/TP/TP2/metnum_tp2/notebooks/calidad_k_alpha
Python 3.8.10


In [3]:
#funcion general del modelo
def predict(X_train, y_train, X_val, alpha, k, con_pca):
    
    clf = metnum.KNNClassifier(k)
    
    if con_pca:
        
        pca = metnum.PCA(alpha)
        pca.fit(X_train)
        
        X_train = pca.transform(X_train)
        X_val = pca.transform(X_val)
        
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    
    return y_pred

In [4]:
import numpy as np

def k_folds(X,y, folds):
    
    fold_size = int(X.shape[0]/folds)
    
    X_folds = []
    y_folds = []
    
    for i in range(folds):
        
        if (i + 1)*fold_size < X.shape[0]: limit = (i + 1)*fold_size
        else: limit = X.shape[0]
        
        X_folds.append(X[i*fold_size:limit])
        y_folds.append(y[i*fold_size:limit])
        
    return X_folds, y_folds

def join_folds(X_list, y_list):
    
    X_join = X_list[0]
    y_join = y_list[0]
    
    for i in range(1,len(X_list)):
    
        X_join = np.append(X_join, X_list[i], axis = 0)
        y_join = np.append(y_join, y_list[i], axis = 0)
        
    return X_join, y_join

In [5]:
# metrics

def compare_value(y_predict, y_val, i):
    
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for j in range(len(y_val)):
        
        if y_val[j][0] == i:
            if y_predict[j][0] == i: tp+=1
            else: fn += 1
        else:
            if y_predict[j][0] == i: fp += 1
            else: tn += 1

    return tp, tn, fp, fn
            
def metrics_value(y_pred, y_val, i):
    
    tp, tn, fp, fn = compare_value(y_pred, y_val, i)
    
    acc = (tp + tn)/(tp + tn + fp + fn)
    prec = tp/(tp + fp)
    rec = tp/(tp + fn)
    
    return acc, prec, rec

def metrics(y_pred, y_val):
    
    acc_sum = 0
    prec_sum = 0
    rec_sum = 0
    
    for i in range(10):
        
        acc, prec, rec = metrics_value(y_pred, y_val, i)
        
        acc_sum += acc
        prec_sum += prec
        rec_sum += rec
        
    return acc_sum/10, prec_sum/10, rec_sum/10

In [6]:
# Extraigo data set MNIST
%load_ext autoreload
%autoreload 2

import pandas as pd

df_train = pd.read_csv("../../data/train.csv")

df_train = df_train.sample(frac = 1) # shuffle df

#df_train = df_train[:5000]

X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

In [7]:
import matplotlib.pyplot as plt
from tqdm import tqdm

## KNN (sin PCA)

In [8]:
#Particiono los datos
limit = int(0.8 * X.shape[0]) 

X_train, y_train = X[:limit], y[:limit]
X_val, y_val = X[limit:], y[limit:]

In [ ]:
k_list = [1,2,5,10,20,50,100]

acc_list = []
prec_list = []
rec_list = []

print("hey")

for k in k_list:
    
    print("k: " + str(k))
    
    y_pred = predict(X_train, y_train, X_val, 0,  k, False)
    y_pred = y_pred.reshape(len(y_pred), 1)
    acc, prec, rec = metrics(y_pred, y_val)
    
    acc_list.append(acc)
    prec_list.append(prec)
    rec_list.append(rec)
    
print("enjoy your data")

hey
k: 1
k: 2
k: 5
k: 10
k: 20


In [ ]:
plt.plot(k_list, acc_list)
plt.ylim(0,1)
plt.show()

In [ ]:
plt.plot(k_list, prec_list)
plt.ylim(0,1)
plt.show()

In [ ]:
plt.plot(k_list, rec_list)
plt.ylim(0,1)
plt.show()

## KNN con PCA

In [ ]:
k_list = [1,2,5,10,20,50,100]
alpha_list = [1,5,10,50,100]

acc_list_pca = []
prec_list_pca = []
rec_list_pca = []

print("hey")

for k in k_list:
    
    for alpha in alpha_list:
        
        print("k: " + str(k) + " alpha: " + str(alpha))
    
        y_pred = predict(X_train, y_train, X_val, alpha,  k, True)
        y_pred = y_pred.reshape(len(y_pred), 1)
        acc, prec, rec = metrics(y_pred, y_val)
        
        acc_list_pca.append(acc)
        prec_list_pca.append(prec)
        rec_list_pca.append(rec)

print("enjoy your data")

In [ ]:
acc_list_pca = np.array(acc_list_pca)
prec_list_pca = np.array(prec_list_pca)
rec_list_pca = np.array(rec_list_pca)

In [ ]:
plt.imshow(acc_list_pca.reshape(len(k_list),len(alpha_list)), vmin = 0, vmax = 1, cmap = "gray")
plt.ylabel("k_value")
plt.xlabel("alpha_values")
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(prec_list_pca.reshape(len(k_list),len(alpha_list)), vmin = 0, vmax = 1, cmap = "gray")
plt.ylabel("k_value")
plt.xlabel("alpha_values")
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(rec_list_pca.reshape(len(k_list),len(alpha_list)), vmin = 0, vmax = 1, cmap = "gray")
plt.ylabel("k_value")
plt.xlabel("alpha_values")
plt.colorbar()
plt.show()